In [1]:
import os
import json
import pandas as pd
from pathlib import Path

In [2]:
result_path = Path(os.getcwd()) / "results"

In [3]:
result_summary = {}
summary_file_name = "summary.json"

In [4]:
for llm in os.listdir(result_path):
    if os.path.isdir(result_path / llm):
        result_summary[llm] = {}
        for task in os.listdir(result_path / llm):
            if os.path.isdir(result_path / llm / task):
                result_summary[llm][task] = {}
                for pe in os.listdir(result_path / llm / task):
                    if os.path.isdir(result_path / llm / task / pe):
                        result_summary[llm][task][pe] = {}
                        files = [file for file in os.listdir(result_path / llm / task / pe) if
                                 file != summary_file_name and file != ".DS_Store"]
                        with open(result_path / llm / task / pe / summary_file_name, "r") as f:
                            result_summary[llm][task][pe] = json.load(f)

In [5]:
result_summary

{'palm-2-chat': {'anli': {'zero-shot': {'model': 'palm-2-chat',
    'task': 'anli',
    'pe_technique': 'zero-shot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 484,
    'total': 1200,
    'accuracy': 40.33333333333333,
    'created_at': '2023-11-25-22-02-34'},
   'null-shot-cot': {'model': 'palm-2-chat',
    'task': 'anli',
    'pe_technique': 'null-shot-cot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 379,
    'total': 1200,
    'accuracy': 31.583333333333336,
    'created_at': '2023-11-26-02-08-54'},
   'zero-shot-cot': {'model': 'palm-2-chat',
    'task': 'anli',
    'pe_technique': 'zero-shot-cot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 265,
    'total': 1200,
    'accuracy': 22.083333333333332,
    'created_at': '2023-11-26-00-44-52'},
   'null-shot': {'model': 'palm-2-chat',
    'task': 'anli',
    'pe_technique': 'null-shot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 

In [6]:
for model in result_summary.keys():
    for task in result_summary[model].keys():
        for pe in result_summary[model][task].keys():
            lengths = []
            time_takens = []
            files = [file for file in os.listdir(result_path / model / task / pe) if file != summary_file_name and file != ".DS_Store"]
            for file in files:
                with open(Path(result_path) / model / task / pe / file) as f:
                    data = json.load(f)
                    lengths.append(data["length"])
                    time_takens.append(data["time_taken"])
            mean_length = sum(lengths) / len(lengths)
            mean_time_taken = sum(time_takens) / len(time_takens)
            std_length = (sum([(length - mean_length) ** 2 for length in lengths]) / len(lengths)) ** 0.5
            std_time_taken = (sum([(time_taken - mean_time_taken) ** 2 for time_taken in time_takens]) / len(time_takens)) ** 0.5
            min_length = min(lengths)
            min_time_taken = min(time_takens)
            max_length = max(lengths)
            max_time_taken = max(time_takens)
            median_length = sorted(lengths)[len(lengths) // 2]
            median_time_taken = sorted(time_takens)[len(time_takens) // 2]
            total_time_taken = sum(time_takens)
            result_summary[model][task][pe]["mean_length"] = mean_length
            result_summary[model][task][pe]["median_length"] = median_length
            result_summary[model][task][pe]["std_length"] = std_length
            result_summary[model][task][pe]["min_length"] = min_length
            result_summary[model][task][pe]["max_length"] = max_length
            result_summary[model][task][pe]["mean_time_taken"] = mean_time_taken
            result_summary[model][task][pe]["median_time_taken"] = median_time_taken
            result_summary[model][task][pe]["std_time_taken"] = std_time_taken
            result_summary[model][task][pe]["min_time_taken"] = min_time_taken
            result_summary[model][task][pe]["max_time_taken"] = max_time_taken
            result_summary[model][task][pe]["total_time_taken"] = total_time_taken            

In [7]:
df = pd.DataFrame.from_dict({(i, j, k): result_summary[i][j][k]
                             for i in result_summary.keys()
                             for j in result_summary[i].keys()
                             for k in result_summary[i][j].keys()},
                            orient='index')

In [8]:
df = df.drop(columns=["current_index", "is_processing"])

In [9]:
df

model        task  \
palm-2-chat   anli       zero-shot        palm-2-chat        anli   
                         null-shot-cot    palm-2-chat        anli   
                         zero-shot-cot    palm-2-chat        anli   
                         null-shot        palm-2-chat        anli   
              strategyqa zero-shot        palm-2-chat  strategyqa   
...                                               ...         ...   
gpt-3.5-turbo aqua       null-shot      gpt-3.5-turbo        aqua   
              race-h     zero-shot      gpt-3.5-turbo      race-h   
                         null-shot-cot  gpt-3.5-turbo      race-h   
                         zero-shot-cot  gpt-3.5-turbo      race-h   
                         null-shot      gpt-3.5-turbo      race-h   

                                         pe_technique  correct  total  \
palm-2-chat   anli       zero-shot          zero-shot      484   1200   
                         null-shot-cot  null-shot-cot      379   1200   
                         zero-shot-cot  zero-shot-cot      265   1200   
                         null-shot          null-shot      492   1200   
              strategyqa zero-shot          zero-shot     1300   2290   
...                                               ...      ...    ...   
gpt-3.5-turbo aqua       null-shot          null-shot      146    254   
              race-h     zero-shot          zero-shot     2859   3498   
                         null-shot-cot  null-shot-cot     2721   3498   
                         zero-shot-cot  zero-shot-cot     2717   3498   
                         null-shot          null-shot     2825   3498   

                                         accuracy           created_at  \
palm-2-chat   anli       zero-shot      40.333333  2023-11-25-22-02-34   
                         null-shot-cot  31.583333  2023-11-26-02-08-54   
                         zero-shot-cot  22.083333  2023-11-26-00-44-52   
                         null-shot      41.000000  2023-11-25-23-20-14   
              strategyqa zero-shot      56.768559  2023-11-20-18-25-26   
...                                           ...                  ...   
gpt-3.5-turbo aqua       null-shot      57.480315  2023-11-19-17-53-24   
              race-h     zero-shot      81.732419  2023-11-23-14-21-54   
                         null-shot-cot  77.787307  2023-11-24-08-07-07   
                         zero-shot-cot  77.672956  2023-11-24-04-44-30   
                         null-shot      80.760435  2023-11-23-23-55-04   

                                        mean_length  median_length  \
palm-2-chat   anli       zero-shot       940.503333            893   
                         null-shot-cot  1287.180833           1200   
                         zero-shot-cot  1037.376667            977   
                         null-shot      1038.062500            965   
              strategyqa zero-shot       877.141921            851   
...                                             ...            ...   
gpt-3.5-turbo aqua       null-shot       598.035433            536   
              race-h     zero-shot        52.949400             40   
                         null-shot-cot   298.867639            310   
                         zero-shot-cot   419.522584            367   
                         null-shot        83.260720             44   

                                        std_length  min_length  max_length  \
palm-2-chat   anli       zero-shot      364.215984           0        3673   
                         null-shot-cot  494.813151           0        4658   
                         zero-shot-cot  375.734924           0        3745   
                         null-shot      424.794947           0        4658   
              strategyqa zero-shot      580.694012           0        5509   
...                                            ...         ...         ...   
gpt-3.5-turbo aqua       null-shot      324.038400          65    

In [10]:
df.to_csv("summary.csv")